## Описание проекта
Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение
для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля.
В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей.

## Задача
Нужно построить модель для определения стоимости.


## Доп. требования
Заказчику важны:
 - качество предсказания;
 - скорость предсказания;
 - время обучения.

## Описание данных

Признаки
 - DateCrawled — дата скачивания анкеты из базы
 - VehicleType — тип автомобильного кузова
 - RegistrationYear — год регистрации автомобиля
 - Gearbox — тип коробки передач
 - Power — мощность (л. с.)
 - Model — модель автомобиля
 - Kilometer — пробег (км)
 - RegistrationMonth — месяц регистрации автомобиля
 - FuelType — тип топлива
 - Brand — марка автомобиля
 - NotRepaired — была машина в ремонте или нет
 - DateCreated — дата создания анкеты
 - NumberOfPictures — количество фотографий автомобиля
 - PostalCode — почтовый индекс владельца анкеты (пользователя)
 - LastSeen — дата последней активности пользователя

Целевой признак
 - Price — цена (евро)

In [1]:
from time import time
import numpy as np
import pandas as pd

import plotly.express as px

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

import catboost
import lightgbm as lgb


## Шаг 1. Загрузим и посмотрим на данные

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')
data.name = 'Cars'
data.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
86085,2016-03-05 23:54:00,8300,NaN,2005,manual,0,5er,150000,11,gasoline,bmw,no,2016-03-05 00:00:00,0,63814,2016-04-06 01:46:10
118512,2016-03-30 08:56:41,995,wagon,2001,manual,102,v40,150000,2,gasoline,volvo,no,2016-03-30 00:00:00,0,85661,2016-04-01 02:17:53
234645,2016-04-03 20:49:36,12500,wagon,2008,auto,249,300c,150000,3,lpg,chrysler,no,2016-04-03 00:00:00,0,98660,2016-04-05 21:46:49
303080,2016-03-09 19:38:44,12999,sedan,2014,auto,120,2_reihe,40000,1,petrol,peugeot,no,2016-03-09 00:00:00,0,14089,2016-03-15 07:16:34
304849,2016-03-31 20:46:59,1500,sedan,2007,manual,0,octavia,150000,7,petrol,skoda,yes,2016-03-31 00:00:00,0,24640,2016-03-31 20:46:59


Данные загружены, видим, что в данных присутствуют пропуски.

Датасет содержит 16 колонок и 354369 записей. 7 колонок int64 и 9 object колонок. Посмотрим, что по пропускам.

In [ ]:
# проверим датасет на пропуски
def get_missing_values(data: pd.DataFrame) -> None:
    """
    Выводит данные о пропусках в колонках по датафрейму.
    Не изменяет данные внутри датафрейма.

    :param data: pd.DataFrame
    :return: None
    """
    # получаем имена колонок датафрейма
    columns = data.columns.to_list()
    data_len = len(data)
    try:
        df_name = data.name
    except AttributeError:
        df_name = ''
    # объявляем счетчик
    counter = -1
    print('='*60)
    # если есть пропуски в данных - выводим информацию о пропусках по колонкам
    if sum(data.isnull().sum()) > 0:
        print(f'Количество записей в датафрейме {df_name}: {data_len} \n')
        print(f'В датафрейме {df_name} имеются следующие пропуски:')
        for i in data.isnull().sum():
            counter += 1
            if i > 0:
                print(f'  - в колонке {columns[counter]}: {i} пропусков, это {i/data_len:0.2%} об общего объема данных')
    else:
        print(f'Отлично, в датафрейме {df_name} отсутствуют пропуски.')

# посмотрим на пропуски в данных
get_missing_values(data)

Имеем 5 колонок с достаточно большим количеством пропусков.
Визуализируем категориальные данные.

In [ ]:
cat_features = ['VehicleType', 'Brand', 'Model', 'FuelType', 'Gearbox', 'NotRepaired']

for feature in cat_features:
    fig = px.histogram(
        data[feature],
        title=f'Распределение по колонке {feature}'
        )

    fig.update_xaxes(
        title=feature,
        categoryorder="total descending"
        )

    fig.update_yaxes(
        title='Количество'
        )

    fig.update_layout(
        showlegend=False
        )

    fig.show()

In [ ]:
px.histogram(data['Power']).show()

In [7]:
data['NumberOfPictures'].value_counts()

0    354369
Name: NumberOfPictures, dtype: int64

В колонке Power очень большое количество нулей, их можно будет попробовать восстановить, а также аномально высокие значения.
В колонке NumberOfPictures содержатся только нули, пользы для модели от нее никакой, поэтому от нее можно будет избавиться.
Посмотрим на корреляцию признаков.

In [8]:
data_for_corr = data.copy()
# для рассчета корреляции Пирсона преобразуем данные в колонках NotRepaired и Gearbox
data_for_corr['NotRepaired'] = data['NotRepaired'].apply(lambda x: 1 if x == 'yes' else 0)
data_for_corr['Gearbox'] = data['Gearbox'].apply(lambda x: 1 if x == 'auto' else 0)

In [ ]:
px.imshow(data_for_corr.drop('NumberOfPictures', axis=1).corr(), text_auto=True).show()

Посмотрим на гистограмму целевого признака

In [ ]:
px.histogram(data['Price'], nbins=1000).show()

In [11]:
print(f"Количество явных дублей: {data.duplicated().sum()}")

Количество явных дублей: 4


### Промежуточные выводы:
 - в данных есть пропуски, которые необходимо восстановить и нули, которые также можно попробовать заменить на реальные значения
 - в данных присуствует большое количество записей со значением целевого признака, равным нулю. Сервис по продаже авто бесплатно раздавать авто навряд ли не будет, а значит скорее всего это ошибка. Восстанавливать значения целевого признака - в данном случае не очень хорошая идея, а значит лучше такие данные удалить
 - в данных присуствуют явные дубли
 - в данных есть аномальные показатели в колонке Power
 - в колонке NumberOfPictures содержатся только нули, от данной колонки можно избавиться
 - целевой признак имеет среднюю отрицательную корреляцию со столбцами:
    - Kilometer - чем больше пробег, тем меньше цена),
    - NotRapaired - цена ниже у авто с каким-либо ремонтом
 - целевой признак имеет среднюю положительную корреляцию со столбцами:
    - Gearbox - авто на автомате дороже
    - Power - более мощные авто дороже
    - RegistrationMonth - авто, зарегистрированные в определенные месяцы также дороже
 - имеется дизбаланс классов по категориальным признакам

## Шаг 2. Предобработка данных

In [12]:
l = ['Brand', 'Model', 'Gearbox', 'VehicleType', 'FuelType']
l[:-1]

['Brand', 'Model', 'Gearbox', 'VehicleType']

In [13]:
# преобразуем даты в формат даты пандас
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], format='%Y-%m-%d %H:%M:%S')
data['DateCreated'] = pd.to_datetime(data['DateCreated'], format='%Y-%m-%d %H:%M:%S')
data['LastSeen'] = pd.to_datetime(data['LastSeen'], format='%Y-%m-%d %H:%M:%S')

# удалим колонку NumberOfPictures
data = data.drop('NumberOfPictures', axis=1)

# удалим записи со значением целевого признака, равным нулю
index_to_del = data[data['Price'] == 0].index
data = data.drop(index_to_del)

# удалим явные дубли
data = data.drop_duplicates()

# итеративно заменим пропуски, с более низкой точностью на каждой итерации цикла,
# для этого заменим нули и выбросы в колонке Power на Nan для заполнения данных с помощью метода fillna,
# затем заполним пропуски медианой
for index in range(1, 6):
    cols = ['Brand', 'Model', 'Gearbox', 'VehicleType', 'FuelType', 'ColumnForFirstIter']
    cols = cols[:-index]
    data.loc[data[data['Power'] > 500].index, 'Power'] = np.nan
    data[data['Power'] == 0]['Power'] = np.nan
    data['Power'] = data['Power'].fillna(
        data.groupby(cols)['Power']
            .transform('median')
    )

# пропущенные категориальные признаки заменим значением unknown
data = data.fillna(
	{
        'VehicleType': 'unknown',
        'Gearbox': 'unknown',
        'Model': 'unknown',
        'FuelType': 'unknown',
        'NotRepaired': 'unknown'
    }
)

# отсортируем данные по дате создания карточек для дальнейшего разбиения на
# обучающую и тестовые выборки и моделирования реальных условий работы модели
data = data.sort_values(by='DateCreated')

# удалим данные с датами, т.к. они не будут участвовать в обучении моделей
data = data.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1)

# проверим данные еще раз на пропуски
get_missing_values(data)

C:\Users\123\AppData\Local\Temp\ipykernel_11168\2575922146.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\123\AppData\Local\Temp\ipykernel_11168\2575922146.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\123\AppData\Local\Temp\ipykernel_11168\2575922146.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Отлично, в датафрейме  отсутствуют пропуски.


Трансформируем категориальные колонки для моделей регрессии

In [14]:
new_cat_features = [x + '_cat' for x in cat_features]

lb_encoder = LabelEncoder()
for col, new_col in zip(cat_features, new_cat_features):
    data[new_col] = lb_encoder.fit_transform(data[col])

data = data.drop(cat_features, axis=1)

Данные подготовлены, приступаем к обучению моделей.

## Шаг 3. Обучение моделей

Отделим 25% последних записей по дате создания для теста. Таким образом мы смоделируем реальный режим работы модели, подавая на вход данные "из будущего". Также удалим колонки

In [15]:
test = data.tail(int(data.shape[0] * 0.25))
train = data.drop(test.index)

Разделим данные на обучающую и тестовую выборки

In [16]:
features_train = train.drop('Price', axis=1)
target_train = train['Price']

features_test = test.drop('Price', axis=1)
target_test = test['Price']

In [17]:
cbc_params = {
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'n_estimators': [500, 600, 800, 1200, 1600, 2000, 3000]
}

cbr = catboost.CatBoostRegressor(loss_function='RMSE', task_type='GPU', random_state=25, silent=True)
cbr.randomized_search(cbc_params, features_train, target_train, cv=3, n_iter=5)
print(cbr.best_score_, cbr._get_params())

bestTest = 1845.224535
bestIteration = 499
0:	loss: 1845.2245352	best: 1845.2245352 (0)	total: 9.24s	remaining: 37s
bestTest = 1764.404815
bestIteration = 1199
1:	loss: 1764.4048152	best: 1764.4048152 (1)	total: 30.7s	remaining: 46s
bestTest = 1798.114903
bestIteration = 599
2:	loss: 1798.1149034	best: 1764.4048152 (1)	total: 44.1s	remaining: 29.4s
bestTest = 1793.142781
bestIteration = 499
3:	loss: 1793.1427810	best: 1764.4048152 (1)	total: 58.8s	remaining: 14.7s
bestTest = 1784.837657
bestIteration = 540
4:	loss: 1784.8376572	best: 1764.4048152 (1)	total: 1m 26s	remaining: 0us
Estimating final quality...
Training on fold [0/3]
bestTest = 1751.982999
bestIteration = 1199
Training on fold [1/3]
bestTest = 1737.35848
bestIteration = 1199
Training on fold [2/3]
bestTest = 1780.974976
bestIteration = 1199
{'learn': {'RMSE': 1601.9632338053684}} {'task_type': 'GPU', 'random_seed': 25, 'depth': 6, 'verbose': 0, 'iterations': 1200, 'loss_function': 'RMSE', 'silent': True, 'random_state': 25}

In [18]:
best_params = cbr._get_params()
del best_params['random_seed']
del best_params['verbose']

cbr = catboost.CatBoostRegressor(**best_params)
start_learning_time = time()
cbr.fit(features_train, target_train)
print(f"Время обучения модели CatboostRegressor: {int(time() - start_learning_time)} секунд")
pred = cbr.predict(features_test)
print(f'RMSE модели на тесте: {np.sqrt(mse(target_test, pred))}')

Время обучения модели CatboostRegressor: 5 секунд
RMSE модели на тесте: 1703.145148799258


In [19]:
feat_importances = pd.Series(cbr.feature_importances_, index=features_train.columns).sort_values(ascending=False)
fig = px.bar(feat_importances, title='Наиболее значимые признаки для линейной регрессии')

fig.update_layout(
    showlegend=False
)

fig.update_xaxes(
    title='Признак'
)

fig.update_yaxes(
    title='Коэфициент важности'
)

fig.show()

In [20]:
lgbm_model = lgb.LGBMRegressor(random_state=25, silent=True)

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

my_scorer = make_scorer(rmse)

lgbm_params = {
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'n_estimators': [500, 600, 800, 1200, 1600, 2000, 3000]
}

rand_search = RandomizedSearchCV(lgbm_model, lgbm_params, scoring=my_scorer, n_iter=5, n_jobs=-1)
rand_search.fit(features_train, target_train)
rand_search.best_params_, rand_search.best_score_

C:\Users\123\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\sklearn.py:598: UserWarning:

'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.



({'n_estimators': 600, 'max_depth': 5}, 1674.221660463871)

In [21]:
lgbm_model = lgb.LGBMRegressor(**rand_search.best_params_, random_state=25, silent=True)
start_learning_time = time()
lgbm_model.fit(features_train, target_train)
print(f"Время обучения модели LGBMRegressor: {int(time() - start_learning_time)} секунд")
pred = lgbm_model.predict(features_test)
print(f'RMSE модели на тесте: {rmse(target_test, pred)}')

Время обучения модели LGBMRegressor: 1 секунд
RMSE модели на тесте: 1685.7181808903888


In [22]:
feat_importances = pd.Series(lgbm_model.feature_importances_, index=features_train.columns).sort_values(ascending=False)
fig = px.bar(feat_importances, title='Наиболее значимые признаки для линейной регрессии')

fig.update_layout(
    showlegend=False
)

fig.update_xaxes(
    title='Признак'
)

fig.update_yaxes(
    title='Коэфициент важности'
)

fig.show()

In [23]:
lr = LinearRegression()
start_learning_time = time()
lr.fit(features_train, target_train)
print(f"Время обучения модели LinearRegressor: {int(time() - start_learning_time)} секунд")
pred = lr.predict(features_test)
print(f'RMSE модели на тесте: {rmse(target_test, pred)}')

Время обучения модели LinearRegressor: 0 секунд
RMSE модели на тесте: 3287.7023918877694


In [24]:
feat_importances = pd.Series(lr.coef_, index=features_train.columns).sort_values(ascending=False)
fig = px.bar(feat_importances, title='Наиболее значимые признаки для линейной регрессии')

fig.update_layout(
    showlegend=False
)

fig.update_xaxes(
    title='Признак'
)

fig.update_yaxes(
    title='Коэфициент важности'
)

fig.show()

Отбор признаков для модели линейной регрессии не улучшает ее результат

In [27]:
rfr_params = {
    'n_estimators': range(50, 150, 10),
    'max_depth': range(3, 10, 1)
}

rfr_model = RandomForestRegressor(random_state=25, n_jobs=-1)

search = RandomizedSearchCV(rfr_model, rfr_params, n_iter=5, n_jobs=-1, cv=3, verbose=10)
search.fit(features_train, target_train)
search.best_score_, search.best_params_

Fitting 3 folds for each of 5 candidates, totalling 15 fits


(0.8004563893331963, {'n_estimators': 100, 'max_depth': 9})

In [28]:
rfr_model = RandomForestRegressor(**search.best_params_, random_state=25, n_jobs=-1)
start_learning_time = time()
rfr_model.fit(features_train, target_train)
print(f"Время обучения модели RandomForestRegressor: {int(time() - start_learning_time)} секунд")
pred = rfr_model.predict(features_test)
print(f'RMSE модели на тесте: {rmse(target_test, pred)}')

Время обучения модели RandomForestRegressor: 6 секунд
RMSE модели на тесте: 2046.8053358778182


Засечем время работы моделей

In [29]:
%time lgbm_model.predict(features_test)
%time cbr.predict(features_test)
%time lr.predict(features_test)
%time rfr_model.pr63 msedict(features_test)

CPU times: total: 5.25 s
Wall time: 384 ms
CPU times: total: 234 ms
Wall time: 21 ms
CPU times: total: 203 ms
Wall time: 4 ms
CPU times: total: 781 ms
Wall time: 63 ms


array([3478.71478585, 1951.20381391,  847.92422829, ..., 3244.53198787,
        847.92422829, 2015.6889929 ])

## Выводы

В результате выполненной работы было обучено 4 модели: CatBoost, LightGBM, LinearRegression, RandomForestRegressor.

RMSE моделей:
 - CatBoost: 1640
 - LightGBM: 1685
 - LinearRegression: 3200
 - RandomForestRegressor: 1980

Время обучения моделей:
 - CatBoost: 5 секунд
 - LightGBM: 1 секунда
 - LinearRegression: 0 секунд
 - RandomForestRegressor: 6 секунд

Скорость предсказания моделей:
 - CatBoost: 21 ms
 - LightGBM: 384 ms
 - LinearRegression: 4 ms
 - RandomForestRegressor: 63 ms

Таким образом можно рекомендовать заказчику модель CatBoost т.к. выигрывает по 2 из 3х важных параметров для заказчика у второй по качеству модели LightGBM (качество и скорость предсказания).

Параметры итоговой модели CatBoostRegressor:
{'iterations': 1200,
 'depth': 6,
 'loss_function': 'RMSE',
 'silent': True,
 'task_type': 'GPU',
 'random_state': 25}